In [1]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 
import queries
from datetime import datetime
import calendar

In [74]:
# using a dates as a range is several times faster, will use ranges and optimize 
date_select = pd.date_range('2022-08-01','2022-09-01',freq='D')
#extract date from datetime and convert to stirng 
dates = [str(datetime.date(i)) for i in dates]

In [ ]:
strftime("%m/%d/%Y, %H:%M:%S")

In [76]:
dates[0] 

'2022-08-01'

In [30]:
dates[0].strftime("%Y-%M-%D")

'2022-00-08/01/22'

In [46]:
range(len(dates)-1) 

range(0, 31)

In [44]:
len(dates)

31

In [58]:
#ampersand to chain filters
#must set limit above expected data or default limt set to 1000
api_url1 = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&$where=session_start_dt between '2022-08-01' and '2022-08-02'"


In [3]:
class GetData():
    #api request
     def __init__(self, endpoint_url):
        print("requesting data")
        self.endpoint = endpoint_url
        self.response = requests.get(url=self.endpoint)
        self.data = self.response.json() 
        print("data request complete")
    

In [90]:
test1 = GetData(api_url1)

requesting data
data request complete


In [91]:
len(test1.data)

65382

In [92]:
df = pd.DataFrame(test1.data)

In [93]:
df.head()

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt
0,-6708946392_11_08012022002203,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:03.000,2022-07-31T16:16:48.000,AT,10.69
1,-6708946391_11_08012022002204,854-29080,LEAVENWORTH ST 2900,CASH,2022-08-01T00:22:04.000,2022-07-31T15:00:00.000,AT,5.5
2,-6708946400_11_08012022002204,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:04.000,2022-07-31T17:16:48.000,AT,4.25
3,-6708946399_11_08012022002205,854-29060,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:05.000,2022-07-31T16:16:48.000,AT,7.69
4,-6708946389_11_08012022002206,854-29120,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:06.000,2022-07-31T16:58:36.000,AT,4.25


## gross paid ammount 

In [122]:
df['gross_paid_amt'] = df['gross_paid_amt'].apply(lambda x: float(x))

In [149]:
#rename to dollar ammount 
round(df.gross_paid_amt).head()

0    11.0
1     6.0
2     4.0
3     8.0
4     4.0
Name: gross_paid_amt, dtype: float64

## session times 

In [153]:
#extract date 
df['session_start_date'] = pd.to_datetime(df['session_start_dt']).dt.date

In [148]:
df.head()

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,session_start_date
0,-6708946392_11_08012022002203,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:03.000,2022-07-31T16:16:48.000,AT,10.69,2022-08-01
1,-6708946391_11_08012022002204,854-29080,LEAVENWORTH ST 2900,CASH,2022-08-01T00:22:04.000,2022-07-31T15:00:00.000,AT,5.50,2022-08-01
2,-6708946400_11_08012022002204,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:04.000,2022-07-31T17:16:48.000,AT,4.25,2022-08-01
3,-6708946399_11_08012022002205,854-29060,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:05.000,2022-07-31T16:16:48.000,AT,7.69,2022-08-01
4,-6708946389_11_08012022002206,854-29120,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:06.000,2022-07-31T16:58:36.000,AT,4.25,2022-08-01


In [152]:
#extract time
df['session_start_time'] = pd.to_datetime(df['session_start_dt']).dt.time

In [156]:
df.head()

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,session_start_date,session_start_time
0,-6708946392_11_08012022002203,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:03.000,2022-07-31T16:16:48.000,AT,10.69,2022-08-01,00:22:03
1,-6708946391_11_08012022002204,854-29080,LEAVENWORTH ST 2900,CASH,2022-08-01T00:22:04.000,2022-07-31T15:00:00.000,AT,5.50,2022-08-01,00:22:04
2,-6708946400_11_08012022002204,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:04.000,2022-07-31T17:16:48.000,AT,4.25,2022-08-01,00:22:04
3,-6708946399_11_08012022002205,854-29060,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:05.000,2022-07-31T16:16:48.000,AT,7.69,2022-08-01,00:22:05
4,-6708946389_11_08012022002206,854-29120,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:06.000,2022-07-31T16:58:36.000,AT,4.25,2022-08-01,00:22:06


In [ ]:
df['session_start_time']

In [ ]:
#seperate date and time 
df['session_start'] = pd.to_datetime(df)